In [1]:
from pulp import *
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

In [4]:
#Read data from Excel
excel_file = pd.ExcelFile("DataModel.xlsx")
df_products = pd.read_excel(excel_file, "Products")
df_resources = pd.read_excel(excel_file, "Resources")
df_useResources = pd.read_excel(excel_file, "UseResources")

In [5]:
#Initialize class
model = LpProblem("MaximiseBeaverCreekPottery", LpMaximize)

In [6]:
#Create indices:
#products = ['Bowls', 'Mugs']
products = [row['Products'] for k, row in df_products.iterrows()]
print(products)

['Bowls', 'Mugs']


In [7]:
#resources = ['Labor', 'Clay'] 
resources = [row['Resources'] for k, row in df_resources.iterrows()]
print(resources)

['Labor', 'Clay']


In [8]:
#Define Parameters of the model

# Profit bowls and mugs"
 
#profit = {'Bowls':40, 'Mugs':50}
profit = {products[k]: row['Profit'] for k , row in df_products.iterrows()}
print(profit)

{'Bowls': 40, 'Mugs': 50}


In [9]:
product_resources = [(i,j) for i in products for j in resources]
use_resources = {product_resources[k]: row['Use'] for k , row in df_useResources.iterrows()}
print(use_resources)

{('Bowls', 'Labor'): 1, ('Bowls', 'Clay'): 4, ('Mugs', 'Labor'): 2, ('Mugs', 'Clay'): 3}


In [10]:
total_resources = {resources[k]: row['Total'] for k , row in df_resources.iterrows()}
print(total_resources)

{'Labor': 40, 'Clay': 120}


In [11]:
#Define Decision Variables
#=============================================================================
x = LpVariable.dicts('x', [i for i in products], lowBound=0, cat='Continuous')

#Define Objective function
model += lpSum(profit[i]*x[i] for i in products)             

#Define Constraints
for j in resources:
    model += lpSum(use_resources[(i,j)]*x[i] for i in products) <= total_resources[j]


In [12]:
#solve Model
model.solve()

1

In [14]:
#status of the solution

print("Model status:", LpStatus[model.status])

#Solution - Objective function 
print("The optimal solution is {}".format(value(model.objective)))

#Shadow prices and slack variables
print("\nSensitivity Analysis\nConstraint\t\t\t\t\tShadow Price\tSlack")
for name, c in list(model.constraints.items()):
    print(name, ":", c, "\t\t", c.pi, "\t\t\t", c.slack)

#solution - Value and reduced costs of variables
for v in model.variables():
    print(v.name, "=", v.varValue, "\tReduced Cost =", v.dj)
    


Model status: Optimal
The optimal solution is 1360.0

Sensitivity Analysis
Constraint					Shadow Price	Slack
_C1 : x_Bowls + 2*x_Mugs <= 40.0 		 16.0 			 -0.0
_C2 : 4*x_Bowls + 3*x_Mugs <= 120.0 		 6.0 			 -0.0
x_Bowls = 24.0 	Reduced Cost = -0.0
x_Mugs = 8.0 	Reduced Cost = -0.0
